In [73]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import os

In [74]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import typing
from typing import List, Dict

In [75]:
driver = webdriver.Chrome()

In [63]:
"""

1. make a dataset
    1. Initializing it with correct column names.
-
2. open webpage
3. extract the list as dict
-
4. write to dataset
5. keep doing this
6. write dataset to CSV

"""

'\n\n1. make a dataset\n    1. Initializing it with correct column names.\n-\n2. open webpage\n3. extract the list as dict\n-\n4. write to dataset\n5. keep doing this\n6. write dataset to CSV\n\n'

In [76]:
import pandas as pd 

In [120]:
class ImmoWeb():
    
    def __init__(self, start_url="https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page=1&orderBy=relevance"):
        self.start_url = start_url
        self.advert_urls = []
        self.advert_details = []
        
    def page_advert_urls(self, page_url, url_piece = "annonce"):
        driver.get(page_url)
        time.sleep(random.uniform(1.0, 2.0))
        for elem in driver.find_elements_by_tag_name("a"):
            link = elem.get_attribute("href")
            if not link is None and url_piece in link:
                self.advert_urls.append(link)

    def readPagination(self):
        """
        readPagination gathers all pages that contain independent ads. The first one is hardcoded and 
        aferwards it formats the URL as a string until no new ads are found.
        
        :return: a list of URLs that link to each page of a search query. 
        """
        # initialize a list for storing all the seperate links to individual ads.
        #all_advert_urls = []
        #advert_urls = []
        counter = 2
        print(self.start_url)
        self.page_advert_urls(self.start_url)
        
        previous_adverts_amount = 0
        adverts_amount = len(self.advert_urls)

        while previous_adverts_amount < adverts_amount:
            previous_adverts_amount = adverts_amount
            
            page_url = f"https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page={counter}&orderBy=relevance"
            self.page_advert_urls(page_url)
            
            adverts_amount = len(self.advert_urls)
            counter += 1
            
    def readAdPage(self):
        """
        Method the reads all the information on the page and writes it to a dict using the key, 
        value pairs used on the website in question. 
    
        :inp_url: get the input URL that we will be gathering information from.
        :return: a dict that stores scraped data
        """
        for advert_url in self.advert_urls:
            # initialize some variables.
            detailsKeys = []
            detailsValues = []
            details = {}
    
            # grab the URL page code
            driver.get(advert_url)
    
            # find the tables in the page and split data in 2 types, keys and values.
            for desc_list in driver.find_elements_by_tag_name("th"):
                if desc_list.text != "":
                    detailsKeys.append(desc_list.text)

            for desc_list in driver.find_elements_by_tag_name("td"):
                if desc_list.text != "":
                    if desc_list.find_elements_by_tag_name("a") == []:
                        if desc_list.find_elements_by_tag_name("span") != []:
                            detail = desc_list.text.split("\n")
                            detailsValues.extend(detail[:-1])
                        else:
                            detailsValues.append(desc_list.text)

            # build a dict containing all gathered data
            for idx, x in enumerate(detailsKeys):
                details[x] = detailsValues[idx]
                
            self.advert_details.append(details)